In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

### Importing data

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

train_data = train_data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test_data = test_data[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

train_data = train_data.dropna()
test_data = test_data.dropna()

### Data processing 

In [3]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

# transforming categorical data

# Sex
encoder.fit(train_data['Sex'])
train_data['Sex'] = encoder.transform(train_data['Sex'])
test_data['Sex'] = encoder.transform(test_data['Sex'])

#Embarked
encoder.fit(train_data['Embarked'])
train_data['Embarked'] = encoder.transform(train_data['Embarked'])
test_data['Embarked'] = encoder.transform(test_data['Embarked'])

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# standardize data if needed (not needed for trees, but can be useful for neural networks)
def standardize():
  for column_name, _ in train_data.iteritems():
      if column_name != 'Survived':
        column_train = train_data[column_name].to_numpy().reshape(-1,1)
        scaler.fit(column_train)
        # train data
        scaled_train = scaler.transform(column_train)
        train_data[column_name] = scaled_train
        # test data
        column_test = test_data[column_name].to_numpy().reshape(-1,1)
        scaled_test = scaler.transform(column_test)
        test_data[column_name] = scaled_test

standardize()
x_train = train_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y_train = train_data['Survived']
x_test = test_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y_test = test_data['Survived']

<ipython-input-4-cf87f16166b6>:7: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, _ in train_data.iteritems():


### Neural network

In [32]:

class NeuralNetwork:
  def __init__(self, input_size, output_size, learning_rate=0.001, nb_epochs=16, batch_size=128):  
    # Parametri mreze
    self.learning_rate = learning_rate
    self.nb_epochs = nb_epochs
    self.batch_size = batch_size
    
    # Parametri arhitekture
    self.nb_input = input_size
    self.nb_hidden1 = 256  # 1st layer number of neurons
    self.nb_hidden2 = 256  # 2nd layer number of neurons
    self.nb_classes = output_size   # dead/alive
    # Sama mreza
    self.w = {
        '1': tf.Variable(tf.random.normal([self.nb_input, self.nb_hidden1], dtype=tf.float64)),
        '2': tf.Variable(tf.random.normal([self.nb_hidden1, self.nb_hidden2], dtype=tf.float64)),
        'out': tf.Variable(tf.random.normal([self.nb_hidden2, self.nb_classes], dtype=tf.float64))
    }

    self.b = {
        '1': tf.Variable(tf.random.normal([self.nb_hidden1], dtype=tf.float64)),
        '2': tf.Variable(tf.random.normal([self.nb_hidden2], dtype=tf.float64)),
        'out': tf.Variable(tf.random.normal([self.nb_classes], dtype=tf.float64))
    }
    # Aktivacione funkcije
    self.activation = {
        '1': tf.nn.relu,
        '2': tf.nn.relu,
        'out': tf.nn.softmax
    }

    self.opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

  def fit(self, x_train, y_train):
    nb_train = len(y_train)
    for epoch in range(self.nb_epochs):
        epoch_loss = 0
        nb_batches = int(nb_train / self.batch_size)
        for i in range(nb_batches):
            x = x_train[i*self.batch_size : (i+1)*self.batch_size]
            y = y_train[i*self.batch_size : (i+1)*self.batch_size]
            y_onehot = tf.one_hot(y, self.nb_classes)

            with tf.GradientTape() as tape:
                _, z_out = self.predict(x)

                loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z_out, labels=y_onehot))

            w1_g, w2_g, wout_g, b1_g, b2_g, bout_g = tape.gradient(loss, [self.w['1'], self.w['2'], self.w['out'], self.b['1'], self.b['2'], self.b['out']])

            self.opt.apply_gradients(zip([w1_g, w2_g, wout_g, b1_g, b2_g, bout_g], [self.w['1'], self.w['2'], self.w['out'], self.b['1'], self.b['2'], self.b['out']]))

            epoch_loss += loss

        # U svakoj epohi ispisujemo prosečan loss.
        epoch_loss /= nb_train
        if(epoch+1)%64 == 0:
          print(f'Epoch: {epoch+1}/{self.nb_epochs}| Avg loss: {epoch_loss:.5f}')


  def predict(self, x):
      l1 = tf.add(tf.matmul(x, self.w['1']), self.b['1'])
      a1 = self.activation['1'](l1)
      l2 = tf.add(tf.matmul(a1, self.w['2']), self.b['2'])
      a2 = self.activation['2'](l2)
      l_out = tf.add(tf.matmul(a2, self.w['out']), self.b['out'])
      out = self.activation['out'](l_out)

      pred = tf.argmax(out, 1)

      return pred, l_out

### Training

In [35]:
nn = NeuralNetwork(len(x_train.columns), y_train.nunique(), nb_epochs=512)
nn.fit(x_train, y_train)

Epoch: 64/512| Avg loss: 0.00649
Epoch: 128/512| Avg loss: 0.00406
Epoch: 192/512| Avg loss: 0.00303
Epoch: 256/512| Avg loss: 0.00319
Epoch: 320/512| Avg loss: 0.00263
Epoch: 384/512| Avg loss: 0.00238
Epoch: 448/512| Avg loss: 0.00234
Epoch: 512/512| Avg loss: 0.00268


### Inference

In [36]:
pred, _ = nn.predict(x_test)

pred_correct = tf.equal(pred, y_test)
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

print(f'Test acc: {accuracy:.3f}')

Test acc: 0.734
